# Churn Dataset 

Objective: For this exercise, our aim is to find out if a customer churns or not

In [8]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
spark = SparkSession.builder.appName('logistic_regression_churn').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip install numpy --user' into the EC2 console.
from pyspark.ml.classification import LogisticRegression

%matplotlib inline
import pandas as pd
pd.options.display.mpl_style = 'default'

OptionError: 'You can only set the value of existing options'

In [3]:
# Read in the CSV data.
data = spark.read.csv('BROADBAND_CHURN_SPSS.csv',inferSchema=True,header=True)

In [4]:
# Print data schema.
data.printSchema()

# Print data columns.
data.columns

root
 |-- FXD_ACCT_KEY: integer (nullable = true)
 |-- CAL_DT_KEY: integer (nullable = true)
 |-- CAL_MTH_KEY: integer (nullable = true)
 |-- MTH_END_FLAG: string (nullable = true)
 |-- OPENING_STATUS: string (nullable = true)
 |-- CLOSING_STATUS: string (nullable = true)
 |-- OPENING_SEGMENT: string (nullable = true)
 |-- CLOSING_SEGMENT: string (nullable = true)
 |-- TECHNOLOGY: string (nullable = true)
 |-- OPENING_BASE: integer (nullable = true)
 |-- ADDITIONS: integer (nullable = true)
 |-- DISCONNECTS: integer (nullable = true)
 |-- CLOSING_BASE: integer (nullable = true)
 |-- GROSS_ADDS: double (nullable = true)
 |-- CHURN: integer (nullable = true)
 |-- MIGRATIONS: double (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- SUBURB: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- POST_CODE: string (nullable = true)
 |-- PYMNT_METHOD: string (nullable = true)
 |-- SEG_KEY: integer (nullable = true)
 |-- HIST_START_DTTM: string (nullable = true)
 |-- HIST_

['FXD_ACCT_KEY',
 'CAL_DT_KEY',
 'CAL_MTH_KEY',
 'MTH_END_FLAG',
 'OPENING_STATUS',
 'CLOSING_STATUS',
 'OPENING_SEGMENT',
 'CLOSING_SEGMENT',
 'TECHNOLOGY',
 'OPENING_BASE',
 'ADDITIONS',
 'DISCONNECTS',
 'CLOSING_BASE',
 'GROSS_ADDS',
 'CHURN',
 'MIGRATIONS',
 'TITLE',
 'SUBURB',
 'CITY',
 'POST_CODE',
 'PYMNT_METHOD',
 'SEG_KEY',
 'HIST_START_DTTM',
 'HIST_END_DTTM',
 'CURR_IND',
 'PROD_KEY',
 'PROD_CD',
 'PROD_DESCR',
 'PROD_GRP_DESCR',
 'PROD_TYPE',
 'PROD_INTERNAL_DESCR',
 'MLY_FEE_INCL_GST']

In [9]:
pd.DataFrame(data.take(5), columns=data.columns).transpose()

,0,1,2,3,4
FXD_ACCT_KEY,212540,215540,213980,151459,198740
CAL_DT_KEY,20190831,20190831,20190831,20190831,20190831
CAL_MTH_KEY,201908,201908,201908,201908,201908
MTH_END_FLAG,Y,Y,Y,Y,Y
OPENING_STATUS,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE
CLOSING_STATUS,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE
OPENING_SEGMENT,RESIDENTIAL,RESIDENTIAL,RESIDENTIAL,RESIDENTIAL,RESIDENTIAL
CLOSING_SEGMENT,RESIDENTIAL,RESIDENTIAL,RESIDENTIAL,RESIDENTIAL,RESIDENTIAL
TECHNOLOGY,UFB,UFB,VDSL,UFB,VDSL
OPENING_BASE,1,1,1,1,1


In [10]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
FXD_ACCT_KEY,19999,160805.29631481573,54198.025444961786,24,234226
CAL_DT_KEY,19999,2.0190831E7,0.0,20190831,20190831
CAL_MTH_KEY,19999,201908.0,0.0,201908,201908
MTH_END_FLAG,19999,None,None,Y,Y
OPENING_STATUS,19999,None,None,ACTIVE,PREACTIVE
CLOSING_STATUS,19999,None,None,ACTIVE,NULL
OPENING_SEGMENT,19999,None,None,ENTERPRISE,SME
CLOSING_SEGMENT,19999,None,None,ENTERPRISE,SME
TECHNOLOGY,19999,None,None,ADSL,VDSL
